In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6363ff90c0e17cc39d32b498cbed99fb5addb9c76cb2aba9d923ee18d79431f7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import unicodedata
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
train = pd.read_csv("/kaggle/input/signate-studentcup2023/train.csv")
test = pd.read_csv("/kaggle/input/signate-studentcup2023/test.csv")

train["flag"] = "train"
test["flag"] = "test"
all_data = pd.concat([train, test], ignore_index=True)

In [5]:
# manufacturer
all_data["manufacturer"] = all_data["manufacturer"].str.lower().apply(lambda x: unicodedata.normalize('NFKC', x))
manufacturer_map = {
    'niѕsan':'nissan',
    'nisѕan':'nissan',
    'subαru':'subaru',
    'toyotа':'toyota',
    'sαturn':'saturn',
    'аcura':'acura',
    'vоlkswagen':'volkswagen',
    'lexuѕ':'lexus',
    'ᴄhrysler':'chrysler',
}
all_data["manufacturer"] = all_data["manufacturer"].replace(manufacturer_map)

# fuel
## 欠損値 train: 1239, test, 1495
## わざと欠損値にしている訳ではなさそう？
all_data["fuel"].fillna(value="Unknown", inplace=True)


# odometer
## 後でvalidationを作るときに直そう

# title_status
## 欠損値 train: 456, test: 229
all_data["title_status"].fillna(value="Unknown", inplace=True)

# size
def pre_size(df: pd.DataFrame):
    size_dict = {
        "fullーsize":"full-size",
        "midーsize":"mid-size",
        "subーcompact":"sub-compact",
        "full−size":"full-size",
        "mid−size":"mid-size"
    }
    df["size"] = df["size"].replace(size_dict)
    return df
all_data = pre_size(all_data)

# type
## 欠損値 train: 456, test: 229
all_data["type"].fillna(value="Unknown", inplace=True)


# state
## 欠損値 train: 3304, test: 3029
all_data["state"].fillna(value="Unknown", inplace=True)

In [6]:
def make_sentence(row):
    row["full_sentence"] = f'This car was sold in {row["region"]} and was manufactured by {row["manufacturer"]}. It is in {row["condition"]} condition and has {row["cylinders"]}. It runs on {row["fuel"]} fuel type. The title status is {row["title_status"]}, and it has a {row["transmission"]} transmission. Additionally, it is a {row["drive"]} drive type and falls under the {row["size"]} category. The body type is {row["type"]}, and the color is {row["paint_color"]}. It was sold in {row["state"]}.'
    return row

In [7]:
tqdm.pandas()
all_data = all_data.progress_apply(make_sentence, axis=1)

  0%|          | 0/55069 [00:00<?, ?it/s]

In [8]:
all_data["full_sentence"][0]

'This car was sold in nashville and was manufactured by bmw. It is in excellent condition and has 6 cylinders. It runs on gas fuel type. The title status is clean, and it has a manual transmission. Additionally, it is a rwd drive type and falls under the mid-size category. The body type is convertible, and the color is orange. It was sold in Unknown.'

In [9]:
sentence_embeddings = model.encode(sentences=all_data["full_sentence"].values, 
                                   batch_size=32,
                                   show_progress_bar=True,
                                   device="cuda",
                                  )

Batches:   0%|          | 0/1721 [00:00<?, ?it/s]

In [10]:
sentence_embeddings.shape

(55069, 384)

In [11]:
all_data[[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]] = 0
all_data[[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]] = sentence_embeddings

/tmp/ipykernel_25/2836261218.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_data[[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]] = 0
/tmp/ipykernel_25/2836261218.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_data[[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]] = 0
/tmp/ipykernel_25/2836261218.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [12]:
all_data[all_data["flag"] == "train"][["id"]+[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]].to_csv("/kaggle/working/train_embeddings.csv", index=False)
all_data[all_data["flag"] == "test"][["id"]+[f"embedding_{i}" for i in range(sentence_embeddings.shape[1])]].to_csv("/kaggle/working/test_embeddings.csv", index=False)